In [1]:
import numpy as np
from lib import OPQ, search, evation, product_Ins, product_idxMember, OSDPQ, load_data
import math
from time import *

In [2]:
Ks = 128
MaxFirsSubspaceNumber = 8
SecondSubspaceNumber = 6
start = 3
step = 1
# 加载数据
base_data, query_data, Top_100 = load_data.load_data("mnist")
# 进行OSD空间分解
osd_base_data, osd_query_data = OSDPQ.OSDPQ(base_data, query_data, times=1)

start_time = time()
print("################## 开始构建索引 ##################")
opq = OPQ.OPQ(M=2, Ks=Ks)
opq.fit(osd_base_data)
# 获取得到不同子空间量化误差的比例
P_errors = opq.get_errors_ratio()
print("====== 第一个子空间的量化误差比例为:",P_errors[0], "第二个子空间的量化误差比例为: ======",P_errors[1])
codes = opq.encode(osd_base_data)
idxMembers = product_idxMember.produce_idxCount_idxMembers(codes[:, 0])
sec_idxMembers = product_idxMember.produce_idxCount_idxMembers(codes[:, 1])
Ints2Codebook_MemberCount, Ints2Codebook_Member = product_Ins.produce_Ints(idxMembers, sec_idxMembers, Ks)
end_time = time()
print("构建索引结构花费了：",(end_time-start_time), "s")

for top in [1, 10]:
    Top = Top_100[:, :top]
    print("############### Computing Top %d recall ###############"%top)
    print("-------> Search by DRQE <-------")
    for N in range(start, MaxFirsSubspaceNumber, step):
        for M in [SecondSubspaceNumber]:
            total_recall = 0
            total_nums = 0
            total_clusters = 0
            for i in range(osd_query_data.shape[0]):
                query_data_ = osd_query_data[i]
                dtable= opq.dtable(query_data_) # 得到我们查询点到各个质点距离的倒序，返回的是索引
                query_clusters = search.search_DRQE(dtable, N, M, P_errors, Ks)
                total_clusters += len(query_clusters)
                recall, counter = evation.Searching_Top100(Ints2Codebook_Member, Top[i], Ints2Codebook_MemberCount, query_clusters)
                total_nums = total_nums + counter
                total_recall = total_recall + recall
            if top ==1:
                print("No. First subspace codebook : ",  N)
                print("No. Second subspace codebook : ",  M)
                print("Total Recall  ", total_recall)
                print("Top1 Recall  ", total_recall/(Top.shape[0]))
                print("==========================================")
            else:
                print("No. First subspace codebook : ",  N)
                print("No. Second subspace codebook : ",  M)
                print("Total Recall  ", total_recall)
                print("Top1 Recall  ", total_recall/(Top.shape[0]*Top.shape[1]))
                print("==========================================")

############# loading mnist dataset......#############
############# Doing OSDPQ Space Decomposition ##################
############# OSDPQ Space Decomposition End ##################
################## 开始构建索引 ##################
M: 2, Ks: 128, code_dtype: <class 'numpy.uint8'>
M: 2, Ks: 128, code_dtype: <class 'numpy.uint8'>
iter: 20, seed: 123
Training the subspace: 0 / 2
Training the subspace: 1 / 2
Encoding the subspace: 0 / 2
Encoding the subspace: 1 / 2
=========== PQ量化误差为: 265873.17271302687 ===========
====== 第一个子空间的量化误差比例为: 0.49644571189208353 第二个子空间的量化误差比例为: ====== 0.5035542881079165
Encoding the subspace: 0 / 2
Encoding the subspace: 1 / 2
构建索引结构花费了： 12.183212995529175 s
############### Computing Top 1 recall ###############
-------> Search by DRQE <-------
No. First subspace codebook :  3
No. Second subspace codebook :  6
Total Recall   97
Top1 Recall   0.8083333333333333
No. First subspace codebook :  4
No. Second subspace codebook :  6
Total Recall   102
Top1 Recall   0.85
